# Running a Multi Objective Optimization Directly in Python

This notebook demonstrates how to:

Write a basic Wrapper in Python and launch a multi objective optimization from Python.
If you wanted to launch it from command line, you would do a similar thing of defining the Wrapper, and then put in your configuration file the information about where the wrapper is, and use [BOA's](../index.rst) CLI tools. See [Running an Experiment from Command Line (Python Wrapper)](example_py_run.rst) for more information.

In [12]:
import pathlib

import yaml
import shutil

import boa
from boa.scripts.moo import Wrapper

In [13]:
# Remove old runs to have a clean slate for this example
old_runs = pathlib.Path().resolve().glob("moo_run*")
for path in old_runs:
    shutil.rmtree(path, ignore_errors=True)

## Loading the Config File

In [14]:
config_path = pathlib.Path().resolve().parent.parent / "boa/scripts/moo.yaml"

Here we can see what the configuration file looks like

In [15]:
with open(config_path, 'r') as f:
    file_contents = f.read()
    print (file_contents)

# MultiObjective Optimization config
optimization_options:
    objective_options:
        objective_thresholds:
            - branin >= -194.3809113151761
            - currin >= -10.521741333278536
            - hartmann6 <= 0.2154494439486514
        objectives:
            - name: branin
              lower_is_better: False
              noise_sd: 0
            - name: currin
              lower_is_better: False
              noise_sd: 0
            - name: hartmann6
              lower_is_better: True
              noise_sd: 0

    experiment:
        name: "moo"
    trials: 100

parameters:
    x0:
        type: range
        bounds: [0, 1]
        value_type: float
    x1:
        type: range
        bounds: [0, 1]
        value_type: float
    x2:
        type: range
        bounds: [0, 1]
        value_type: float
    x3:
        type: range
        bounds: [0, 1]
        value_type: float
    x4:
        type: range
        bounds: [0, 1]
        value_type: float
    x5:
    

In [16]:
config = boa.load_jsonlike(config_path)

## Define Our Wrapper

We define our wrapper in wrapper.py and use a synthetic function that stands in for any black box model call

In [17]:
with open(Wrapper.path(), 'r') as f:
    file_contents = f.read()
    print (file_contents)

from pathlib import Path

import numpy as np
import torch

from boa.controller import Controller
from boa.metrics.synthetic_funcs import get_synth_func
from boa.utils import torch_device
from boa.wrappers.base_wrapper import BaseWrapper

tkwargs = {
    "device": torch_device(),
}
Problem = get_synth_func("BraninCurrin")

problem = Problem(negate=True).to(**tkwargs)
hartmann6 = get_synth_func("hartmann6")


class Wrapper(BaseWrapper):
    def run_model(self, trial) -> None:
        pass

    def set_trial_status(self, trial) -> None:
        trial.mark_completed()

    def fetch_trial_data(self, trial, metric_properties, metric_name, *args, **kwargs):
        evaluation = problem(torch.tensor([trial.arm.parameters["x0"], trial.arm.parameters["x1"]]))
        a = float(evaluation[0])
        b = float(evaluation[1])
        c = hartmann6(
            np.array(
                [
                    trial.arm.parameters["x0"],
                    trial.arm.parameters["x1"],
              

## Initialize our Setup

In [18]:
controller = boa.Controller(config_path=config_path, wrapper=Wrapper)

controller.initialize_scheduler()

[INFO 07-09 16:56:56] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='x0', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x1', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x2', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x3', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x4', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x5', parameter_type=FLOAT, range=[0.0, 1.0])], parameter_constraints=[]).
[INFO 07-09 16:56:56] ax.modelbridge.dispatch_utils: Using Models.MOO since there are more ordered parameters than there are categories for the unordered categorical parameters.
[INFO 07-09 16:56:56] ax.modelbridge.dispatch_utils: Calculating the number of remaining initialization trials based on num_initialization_trials=None max_initialization_trials=None num_tunable_parameters=6 num_trials=None use_batch_trials=False
[INFO 07-09 16:56:56] ax.modelbridge.dispatch_

(Scheduler(experiment=Experiment(moo), generation_strategy=GenerationStrategy(name='Sobol+MOO', steps=[Sobol for 12 trials, MOO for subsequent trials]), options=SchedulerOptions(max_pending_trials=10, trial_type=<TrialType.TRIAL: 0>, batch_size=None, total_trials=None, tolerated_trial_failure_rate=0.5, min_failed_trials_for_failure_rate_check=5, log_filepath=None, logging_level=20, ttl_seconds_for_trials=None, init_seconds_between_polls=1, min_seconds_before_poll=1.0, seconds_between_polls_backoff_factor=1.5, timeout_hours=None, run_trials_in_batches=False, debug_log_run_metadata=False, early_stopping_strategy=None, global_stopping_strategy=None, suppress_storage_errors_after_retries=False)),
 <boa.scripts.moo.Wrapper at 0x15546aec0>)

## Run our Experiment

The Controller will save our scheduler to JSON after it completes the run so we can reload it at a later time for analysis or to resume our experiment

In [9]:
scheduler = controller.run()

[INFO 2023-07-09 16:17:54,917 MainProcess] boa: 

##############################################


BOA Experiment Run
Output Experiment Dir: /Users/madelinescyphers/Documents/projs_.nosync/boa/docs/examples/moo_20230709T161752
Start Time: 20230709T161754
Version: 0.8.3.dev1+g0636f51.d20230411

##############################################

[INFO 07-09 16:17:54] Scheduler: Running trials [0]...
[INFO 07-09 16:17:55] Scheduler: Running trials [1]...
[INFO 07-09 16:17:55] Scheduler: Running trials [2]...
[INFO 07-09 16:17:56] Scheduler: Running trials [3]...
[INFO 07-09 16:17:58] Scheduler: Running trials [4]...
[INFO 07-09 16:17:59] Scheduler: Running trials [5]...
[INFO 07-09 16:17:59] Scheduler: Running trials [6]...
[INFO 07-09 16:18:00] Scheduler: Running trials [7]...
[INFO 07-09 16:18:01] Scheduler: Running trials [8]...
[INFO 07-09 16:18:02] Scheduler: Running trials [9]...
[INFO 07-09 16:18:03] Scheduler: Retrieved COMPLETED trials: 0 - 9.
[INFO 07-09 16:18:03] Scheduler: Fetchi

## Get the Best Trial and Output All Trials

In [10]:
trial = scheduler.best_raw_trials()
trial

In [11]:
boa.scheduler_to_df(scheduler)

,trial_index,arm_name,trial_status,generation_method,branin,currin,hartmann6,is_feasible,x0,x1,x2,x3,x4,x5
0,0,0_0,COMPLETED,Sobol,-36.634632,-9.819938,-0.148712,True,0.148094,0.363476,0.832613,0.256167,0.894928,0.794160
1,1,1_0,COMPLETED,Sobol,-112.778679,-5.320959,-0.054360,True,0.696158,0.732111,0.015850,0.195221,0.615510,0.840452
2,2,2_0,COMPLETED,Sobol,-4.082886,-11.484652,-0.777353,False,0.536404,0.029499,0.132064,0.263329,0.070673,0.622516
3,3,3_0,COMPLETED,Sobol,-26.213963,-8.252074,-0.034296,True,0.650621,0.354788,0.427290,0.467677,0.811935,0.700294
4,4,4_0,COMPLETED,Sobol,-106.169212,-5.194883,-0.068215,True,0.040029,0.369770,0.847179,0.403704,0.845237,0.908341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,95_0,COMPLETED,MOO,-5.119286,-7.364519,-1.556391,True,0.154475,0.617918,0.489384,0.266104,0.315524,0.664733
96,96,96_0,COMPLETED,MOO,-89.567719,-6.709397,-2.656048,True,0.062193,0.358119,0.489988,0.262868,0.317533,0.672615
97,97,97_0,COMPLETED,MOO,-33.107964,-9.333147,-2.665807,True,0.142431,0.395321,0.493059,0.262026,0.313797,0.672195
98,98,98_0,COMPLETED,MOO,-28.493637,-1.299150,-0.437964,True,0.000000,0.881082,0.530632,0.267201,0.332852,0.589326
